In [2]:
#Import libraries and set paths
import sys

selenium_path = r'C:\Users\Kimbe\anaconda3\Lib\site-packages\selenium'
PYSOLR_PATH = r'c:\users\kimbe\appdata\local\programs\python\python312\lib\site-packages'

if not PYSOLR_PATH in sys.path:
    sys.path.append(PYSOLR_PATH)

if not selenium_path in sys.path:
    sys.path.append(selenium_path)

import pandas
import numpy
import requests
import bs4
import re
import undetected_chromedriver as uc
import selenium
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time


In [3]:
start_day_nbr = 1
start_month_nbr = 1
start_year_nbr = 2023
start_date = str(start_month_nbr) + '%2F' + str(start_day_nbr) + '%2F' + str(start_year_nbr)
case_data_text = []

#URL to query cases in St. Louis County (aka the 21st Circuit Court)
casenet_url = 'https://www.courts.mo.gov/cnet/searchResult.do?countyCode=SLC&newSearch=Y&courtCode=CT21&startDate=' + start_date + '&caseStatus=A&caseType=All&locationCode='

In [4]:
print(casenet_url)

https://www.courts.mo.gov/cnet/searchResult.do?countyCode=SLC&newSearch=Y&courtCode=CT21&startDate=1%2F1%2F2023&caseStatus=A&caseType=All&locationCode=


In [27]:
#Launch the headless browser
driver = uc.Chrome()

#Navigate to the CaseNet URL
driver.get(casenet_url)
driver.maximize_window()
driver.execute_script("document.body.style.zoom='70%'")

#Give the page ample time to load
time.sleep(10)
wait = WebDriverWait(driver, 10)

#Scroll to ensure the "Next" button is visible
element = driver.find_element(By.ID, "searchResult_next")
actions = ActionChains(driver)
actions.move_to_element(element).perform()

#Wait for the page to load
time.sleep(2) 
    

In [28]:
#Loop through all pages of the table. Find all table rows, and extract all table data, saving the data in a list
while True:
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "searchResult_next"))).click()
        time.sleep(5) # waiting for page loading
        html_content = driver.page_source
        soup = bs4.BeautifulSoup(html_content, 'html.parser')
        
        #Get all table rows on the page
        trs = soup.find_all(attrs={"role": "row"}) 
    
        for tr in trs:
            tds = tr.find_all('td') # output data by referring to the list items by position
            case_data_text.append(tds)
            print(f'Next------------------------------------') # for clarity

    except TimeoutException:
        break
    else:
        numpy.nan
        #driver.close()    



KeyboardInterrupt: 

In [17]:
cases_df = pandas.DataFrame({'CASES_HTML':case_data_text})

In [11]:
file_name = "{:02d}".format(start_year_nbr) + '-' + "{:02d}".format(start_month_nbr) + '-' + "{:02d}".format(start_day_nbr)+'_case_list.csv'

In [12]:
cases_df.to_csv("C:\\Users\\Kimbe\\Documents\\Data Science\\" + file_name, index = False)